In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
from skimage.transform import resize

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *
from conversion import images_to_volume

Setting environment for AWS compute node


No vtk


In [6]:
downsample = 1
# downsample = 32
# downsample=8 is the maximum. Exceeding it results in memory error.

In [7]:
# for stack in all_nissl_stacks:
for stack in ['MD635']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    images = {}
    for sec in metadata_cache['valid_sections'][stack]:
        print sec
        mask = DataManager.load_thumbnail_mask_v3(stack=stack, section=sec, version='aligned_cropped')
    
        if downsample == 32:
            img_rgb = DataManager.load_image(stack, section=sec, resol='thumbnail', version='cropped_tif')
            img = img_as_ubyte(rgb2gray(img_rgb))
            img[~mask] = 0
        else:
            img_rgb = DataManager.load_image(stack, section=sec, resol='lossless', version='cropped_gray')
            if downsample == 1:
                img = img_as_ubyte(rgb2gray(img_rgb)) 
            else:
                img = img_as_ubyte(rgb2gray(img_rgb[::downsample, ::downsample]))
                mask = resize(mask, img.shape[:2]).astype(np.bool)
            img[~mask] = 0
        
        images[sec-1] = img

    voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downsample)
    
#     volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)
    volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=first_sec-1+10)
    print volume.shape
    
#     vol_fp = DataManager.get_intensity_volume_filepath(stack=stack, downscale=downsample)
#     create_if_not_exists(os.path.dirname(vol_fp))
#     bp.pack_ndarray_file(volume, vol_fp)
#     upload_to_s3(vol_fp)
    
#     bbox_fp = DataManager.get_intensity_volume_bbox_filepath(stack=stack, downscale=downsample)
#     np.savetxt(bbox_fp, np.array(volume_bbox, np.int)[None, :], fmt='%d')
#     upload_to_s3(bbox_fp)

MemoryError: 

In [5]:
display_image(volume[200,:,:])

/shared/MouseBrainAtlas/reconstruct/tmp.jpg